# Symposium on Molecular Interactions Tutorials
## What is this?? (Session 2).
This is a Jupyter notebook. It runs Python, but also allows it to be interspersed with Markdown-formatted text, for a nice mixture of documentation and code.

The particular material below is Session 2 of the tutorials on free energy calculations and OpenMM, from Day 1 of the Symposium on Molecular Interactions (2017).

This content is available on the [symposium github repository](https://github.com/QCMM/workshop2017) at [https://github.com/QCMM/workshop2017](https://github.com/QCMM/workshop2017).

This material is a set of slides intended for presentation with RISE (for presentations from Jupyter notebooks). While it may be useful without RISE, it will also likely appear somewhat less verbose than it would if it were intended for use in written form.

## Authors/sources/acknowledgments
Author: David L. Mobley (UCI)

### Sources
- Much of content adapted from materials I have developed or am developing for my [Drug Discovery Computing Techniques class](https://github.com/mobleylab/drug-computing).

### Acknowledgments
- John Chodera and Nathan Lim for help with OpenMM
- OpenEye for help with OpenEye software, especially James Haigh

# Outline of this afternoon's second session
1. Some brief OpenMM basics (what's going on under the hood in Yank)
2. A simple example of a density calculation which stops when converged
3. Analyzing density results in this notebook
4. SMIRNOFF force fields, modifying the density calculation to use them

**To follow along**/run code: Open this Jupyter notebook in `solvation_free_energies/Session2.ipynb` via `jupyter notebook Session2.ipynb`. (See [prerequisites](../prerequisites.md) if you aren't using preinstalled software.) This is cumulative, building on `Session1.ipynb`.



# Generate some input files we'll use

- We want some inputs to work with as we see how OpenMM works
- And for our sample density calculation
- [`SolvationToolkit`](https://github.com/mobleylab/SolvationToolkit) provides simple wrappers for building arbitrary mixtures
  - OpenEye toolkits
  - [`packmol`](https://github.com/mcubeg/packmol)
  - GAFF small molecule force field (and TIP3P or TIP4P, etc. for water)

Let's build a system to use later

In [39]:
from solvationtoolkit.solvated_mixtures import *
mixture = MixtureSystem('mixtures')
mixture.addComponent(label='phenol', smiles='c1ccc(cc1)O', number=1)
mixture.addComponent(label='toluene', smiles='Cc1ccccc1', number=10)
mixture.addComponent(label='cyclohexane', smiles='C1CCCCC1', number=100)
#Generate output files for AMBER
mixture.build(amber = True)

/Users/dmobley/anaconda3/envs/workshop_test/lib/python3.6/site-packages/pandas/io/parsers.py:2218: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/Users/dmobley/anaconda3/envs/workshop_test/lib/python3.6/site-packages/pandas/io/parsers.py:2220: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())



# Mixture 

tolerance 2.000000
filetype pdb
output /var/folders/41/7d3rylg503n8nt_tn0hz6v8c0000gn/T/tmplhm4xyx2/tmp9yz9jwdt.pdb
add_amber_ter


structure /var/folders/41/7d3rylg503n8nt_tn0hz6v8c0000gn/T/tmplhm4xyx2/tmpn4thi8dz.pdb
  number 1 
  inside box 0. 0. 0. 25.521266 25.521266 25.521266
end structure

structure /var/folders/41/7d3rylg503n8nt_tn0hz6v8c0000gn/T/tmplhm4xyx2/tmp7j1f1b2x.pdb
  number 10 
  inside box 0. 0. 0. 25.521266 25.521266 25.521266
end structure

structure /var/folders/41/7d3rylg503n8nt_tn0hz6v8c0000gn/T/tmplhm4xyx2/tmp4ntd_h95.pdb
  number 100 
  inside box 0. 0. 0. 25.521266 25.521266 25.521266
end structure


source leaprc.gaff
source oldff/leaprc.ff99SB
ZSF = loadmol2 in0.mol2
ZFO = loadmol2 in1.mol2
ZQE = loadmol2 in2.mol2
box = loadPdb tbox.pdb
loadamberparams in0.frcmod
loadamberparams in1.frcmod
loadamberparams in2.frcmod
setbox box centers
saveAmberParm box out.prmtop out.inpcrd
quit



# OpenMM is more of a simulation toolkit than a simulation package
- Easy-to-use Python API
- Very fast calculations on GPUs (but slow on CPUs)
- Really easy to implement new techniques, do new science
- Key ingredients in a calculation:
    - `Topology`
    - `System`
    - `Simulation` (takes `System`, `Topology`, `Integrator`; contains positions)

## `Topology`: Chemical composition of your system

- Atoms, bonds, etc.
- Can be loaded from some common file formats such as PDB, mol2
- Can be created from OpenEye molecule via [`oeommtools`](https://github.com/oess/oeommtools), such as [`oeommtools.utils.oemol_to_openmmTop`](https://github.com/oess/oeommtools/blob/master/oeommtools/utils.py#L17)
    - Side note: An OE "molecule" can contain more than one molecule, so can contain protein+ligand+solvent for example
- Tangent: Try to retain bond order info if you have it (e.g. from a mol2)

In [4]:
# Example Topology generation from a couple mechanisms:

# Load a PDB
from simtk.openmm.app import PDBFile
pdb = PDBFile('sample_files/T4-protein.pdb')
t4_topology = pdb.topology

In [35]:
# Load a mol2: MDTraj supports a variety of file formats including mol2
import mdtraj
traj = mdtraj.load('sample_files/mobley_20524.mol2')
# MDTraj objects contain a Topology, but an MDTraj topology; they support conversion to OpenMM
traj.topology.to_openmm()

# MDTraj can also handle PDB, plus trajectory formats which contain topology information
protein_traj = mdtraj.load('sample_files/T4-protein.pdb')
t4_topology = protein_traj.topology.to_openmm()
# And we can visualize with nglview (or drop out to VMD)
import nglview
view = nglview.show_mdtraj(protein_traj)
view

/Users/dmobley/anaconda3/envs/workshop_test/lib/python3.6/site-packages/pandas/io/parsers.py:2218: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/Users/dmobley/anaconda3/envs/workshop_test/lib/python3.6/site-packages/pandas/io/parsers.py:2220: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())


NGLWidget()

In [22]:
# Load AMBER gas phase topology
from simtk.openmm.app import *
prmtop = AmberPrmtopFile('sample_files/mobley_20524.prmtop')
print("Topology has %s atoms" % prmtop.topology.getNumAtoms())

# Gromacs files can be loaded by GromacsTopFile and GromacsGroFile but you need topology/coordinate files
# which don't have include statements, or a GROMACS installation

Topology has 13 atoms


In [36]:
# Load an OEMol and convert (note advantage over MDTraj for bond order, etc.)

from openeye.oechem import *
from oeommtools.utils import *
mol = OEMol()
istream = oemolistream( 'sample_files/mobley_20524.mol2')
OEReadMolecule(istream, mol)
istream.close()

# Convert OEMol to Topology using oeommtools -- so you can get a topology from almost any format OE supports
topology, positions = oemol_to_openmmTop(mol)
print(topology.getNumAtoms())

13


## `System`: Your parameterized system
- Often generated by `createSystem`, but requires OpenMM know how to assign parameters
  - Easy for standard biomolecules (proteins, nucleic acids), waters ions
  - OpenMM FFXML files used; available for many common FFs
  - More complex for general small molecules
- Can also be loaded from common file formats such as GROMACS, AMBER
  - useful if you set up for AMBER or GROMACS
- We have new open forcefield effort that provides new force fields with an `openforcefield.createSystem` operator; generates OpenMM Systems.

In [37]:
# Example system creation
#From OpenMM Docs: http://docs.openmm.org/latest/userguide/application.html#running-simulations
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout

# Example System creation using OpenMM XML force field libraries -- good for biomolecules, ions, water
pdb = PDBFile('sample_files/input.pdb')
forcefield = ForceField('amber99sb.xml', 'tip3p.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, constraints=HBonds)


In [49]:
# Or you could set up your own molecule for simulation with e.g. GAFF using AmberTools

from openmoltools.amber import *
# Generate GAFF-typed mol2 file and AMBER frcmod file using AmberTools
gaff_mol2_file, frcmod_file = run_antechamber('phenol', 'sample_files/mobley_20524.mol2')
# Generate AMBER files
prmtop_name, inpcrd_name = run_tleap( 'phenol', gaff_mol2_file, frcmod_file)
print("Generated %s and %s" % (prmtop_name, inpcrd_name))

# Create System -- in this case, single molecule in gas phase
prmtop = AmberPrmtopFile( prmtop_name)
inpcrd = AmberInpcrdFile( inpcrd_name)
system = prmtop.createSystem(nonbondedMethod = NoCutoff, nonbondedCutoff = NoCutoff, constraints = HBonds)

Generated /Users/dmobley/github/workshop2017/Free_energy_day1/OpenMM/phenol.prmtop and /Users/dmobley/github/workshop2017/Free_energy_day1/OpenMM/phenol.inpcrd


In [58]:
# Load the mixture we generated above in Section 3
file_prefix = 'mixtures/amber/phenol_toluene_cyclohexane_1_10_100'
prmtop = AmberPrmtopFile( file_prefix+'.prmtop')
inpcrd = AmberInpcrdFile( file_prefix+'.inpcrd')
# Create system: Here, solution phase with periodic boundary conditions and constraints
system = prmtop.createSystem(nonbondedMethod = PME, nonbondedCutoff = 1*nanometer, constraints = HBonds)

#You can visualize the above with VMD, or we can do:
traj = mdtraj.load( file_prefix + '.inpcrd', top = file_prefix + '.prmtop')
view = nglview.show_mdtraj(traj)
view

Exception ignored in: <bound method PDBReporter.__del__ of <simtk.openmm.app.pdbreporter.PDBReporter object at 0x124d9aac8>>
Traceback (most recent call last):
  File "/Users/dmobley/anaconda3/envs/workshop_test/lib/python3.6/site-packages/simtk/openmm/app/pdbreporter.py", line 98, in __del__
    if self._topology is not None:
AttributeError: 'PDBReporter' object has no attribute '_topology'


NGLWidget()

## `Simulation`: The system, topology, and positions you're simulating, under what conditions
- Could be for energy minimization, or different types of dynamics
- Has an integrator attached (even if just minimizing), including temperature
- `context` -- including positions, periodic box if applicable, etc.
- If dynamics, has:
  - timestep
  - velocities
- potentially also has reporters which store properties like energies, trajectory snapshots, etc.


### Let's take that last `System` we set up and energy minimize it
(The mixture of toluene, phenol, and cyclohexane we generated originally)

In [84]:
# Prepare the integrator
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)

In [85]:
# Prep the simulation
simulation = Simulation(prmtop.topology, system, integrator)
simulation.context.setPositions(inpcrd.positions)

In [86]:
# Get and print initial energy
state = simulation.context.getState(getEnergy = True)
energy = state.getPotentialEnergy() / kilocalories_per_mole
print("Energy before minimization (kcal/mol): %.2g" % energy)

# Energy minimize
simulation.minimizeEnergy()

# Get and print final energy
state = simulation.context.getState(getEnergy=True, getPositions=True)
energy = state.getPotentialEnergy() / kilocalories_per_mole
print("Energy after minimization (kcal/mol): %.2g" % energy)

Energy before minimization (kcal/mol): 1.4e+11
Energy after minimization (kcal/mol): -5.2e+02


In [87]:
# While we're at it, why don't we just run a few steps of dynamics
simulation.reporters.append(PDBReporter('sample_files/mixture_output.pdb', 100))
simulation.reporters.append(StateDataReporter(stdout, 100, step=True,
        potentialEnergy=True, temperature=True))
simulation.step(1000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
100,-1109.9224637128173,54.69351014571072
200,-192.39539348190215,101.4504354837445
300,543.3308916038798,139.8896963751966
400,1086.4798234975003,174.20000188340617
500,1573.0376620142476,199.48045687227065
600,1889.7272743018164,217.1304905596919
700,2018.7855834912837,233.2740020370527
800,2353.587298148478,243.50956474431544
900,2470.443812347392,256.9557828532118
1000,2675.8771517933874,267.2388578722047


# A simple example of a density calculation which stops when converged


## There's a script version of this which has equilibration, etc... deposited ...

# Analyzing density results in this notebook

# SMIRNOFF force fields, modifying the density calculation to use them

In [5]:
import pandas